# Large application structure

|-flasky
  |-app/
    |-templates/
    |-static/
    |-main/
      |-\_\_init\_\_.py
      |-erorrs.py
      |-forms.py
      |-views.py
    |-\_\_init\_\_.py
    |-email.py
    |-models.py
  |-migrations/
  |-tests/
    |-\_\_init\_\_.py
    |-test\*.py
  |-venv/
  |-requirements.txt
  |-config.py
  |-manage.py

## Configuration Options

- Applications often need several configuration sets. The best example of this is the need to use different databases during development, testing, and production so that they don't interfere with each other.

## Application Package

- The application package is where all the application code, templates, and static files live. It is called simply *app*, though it can be given an application-specific name if desired.

### Using an Application Factory

- The way the application is created in hte single-file version is very convenient, but it has one big drawback. Because the application is created in the global scope, there is no way to apply configuration changes dynamically: by the time the script is running, the application instance has already been created, soit is already too late to make configuration changes. This is particularly important for unit tests because sometimes it is necesarry to run the application under different configuration settings for better test coverage.

- The solution to this problem is to delay the creation of the application by moving it inot a *factory function* that can be explicitly invoked from the script.

- This constructor imports most of the Flask extensions currently in use, but because there is no application instance to initialize them with, it creates them uninitialized by passing no arguments into their constructors. The create_app() function is the application factory, which takes as an argument the name of a configuration to use for the application. The configuration settings stored in one of the classes defined in config.py can be imported directly into the application using the from_object() method available in Flask’s app.config configuration object. The configuration object is selected by name from the config dictionary. Once an application is created and configured, the extensions can be initialized. Calling init_app() on the extensions that were created earlier completes their initialization. 

- The factory function returns the created application instance, but note that applications created with the factory finction in its current state are incomplete, as they are missing routes and custom error page handlers.

### Implementing Application Functionality in a Blueprint 

- The conversion to an application factory introduces a complication for routes. In single-script applications, the application instance exists in the global scope, so routes can be easily defined using the app.route decorator. But now that the application is created at runtime, the app.route decorator begins to exist only after created_app() is invoked, which is too late. Like routes, custom error page handlers persent the same problem, as these are defined with the app.errorhandler decorator.

- Luckiy Flask offers a better solution using blueprints. A blueprint is similar to an application in that it can also define routes. The difference is that routes associated with a blueprint are in a dormant state until hte blueprint is registered with an application, at which point the routes become part of it. Using a blueprint defined in the global scope, the routes of the application can be defined in almost the same way as in the single-script application.

- Like applications, blueprints can be defined all in a single file or can be created in a more structured way with multiple modules inside a package. To allow for the greatest flexibility, a subpackage inside the application package will be created to host the blueprint.

- Blueprints are created by instantiating an object of class Blueprint. The construtor for this class takes two required arguments: the blueprint name and the module or package where the blueprint is located. As with applications Python's \_\_name\_\_ variable is in most cases the correct value for the second argument.

- The routes of the application are stored in an *app/main/view.py* module inside the package, and the error handlers are in *app/main/error.py*. Importing these modules causes the routes and error handlers to be associated with the blueprint. It is important to note that the modules are ipoerted at the bottom of the *app/\_\_init\_\_.py* script to avoid circular dependencies, because views.py and error.py need to import the main blueprint.

> app/main/errors.py:Blueprint with error handlers

- A difference when writing error handlers inside a blueprint is that if the errorhandler decorator is used, the handler will only be invoked for errors that originate in the blueprint. To install application-wide error handlers, the app_errorhandler must be used instead.

> app/main/views.py:Blueprint with application routes

- There are two main differences when writing a view function inside a blueprint . First, as was done for error handlers earlier, the route decorator comes from the blueprint. The second difference is in the usage of the url_for() function. As you may recall, the first argument to this function is the endpoint name of the route, which for application-based routes defaults to the name of the view function. For example, in a single-script application the URL for an index() ivew function can be obtained with url_for('index').

- The difference with blueprint is that Flask applies a namespace to all the endpoints coming from a blueprint so that multiple blueprints can define view functions with the same endpoint named without collisions. The namespace is the name of the blueprint (the first argument to the Blueprint constructor), so the index() view function is registered with endpoint name main.index and its URL can be obtained with url_for('main.index').

- The url_for() function also supports a shorter format for endpoints in blueprints in which the blueprint name is omitted, such as url_fo ('.index'). With this notation, the blueprint for the current request is used. This effectively means that redirects within the same blueprint can use the shorter form, while redirects across blueprints must use the namespaced endpoint name.

- To complete the changes to the application page, the form objects are also stored inside the blueprint in an app/main/forms.py module.

## Launch Script

- The *manage.py* file in the top-level folder is used to start the application.

- The script begins by creating an application. The configuration used is taken from the environment variable FLASK_CONFIG if it's defined; if not, the default configuration is used. 

- As a convenience, a shebang line is added.

## Requirements File

- Applications must include a *requirements.txt* file that records all the packages dependencies, with the exact version numbers. This is important in case the virtual environment needs to be regenerated in a different machine...

## Unit Tests

- This application is very small so there isn't a lot to test yet, but try to examples.

- The tests are written using the standard unittest package from the Python standard library. The setUp() and tearDown() methods run before and after each test, and any methods that have a name that begins with test\_ are executed as tests.

- The setUp() method tries to create an environment for the test that is close to that of a running application. It first creates an application configured for testing and activities its context. This step ensures that tests have access to current_app, like regular requests. Then it creates a brand-new database that the test can use when necessary. The database and the application context are removed in the tearDown() method.

- The first test ensures that the application instance exists. The second test ensure that the application is running under the testing configuration. To make the *tests* folder a proper package, a *tests/\_\_init\_\_.py* file needs to be added, but this can be an empty file, as the unittest package can scan all the modules and locate the tests.

> manage.py: Unit test launcher command

- The manager.command decorator makes it simple to implement custom commands. The name of the decorated function is used as the command name, and the function 's docstring is displayed in the help messages. The implementation of test() function invokes the test renner from the unittest package.

# P. 104